# Create bernCrop Dataset



In [ ]:
# load required modules
import cv2
import eodal
import os
import numpy as np
import h5py
import pandas as pd
import utm
import torch

from pathlib import Path
from eodal.core.sensors import Sentinel2
import geopandas as gpd
from shapely.geometry import Polygon
from eodal.config import get_settings

# make plots larger by default
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 15]

print('eodal version: {}'.format(eodal.__version__))

# we need to tell EOdal that we work using a local data source
settings = get_settings()
settings.USE_STAC = False


output_shapefile_path_BERN = f"../raw_data/LANDKULT/data/BERN_big_bbox.shp"
output_shapefile_path_tiles =f"../raw_data/LANDKULT/data/tiles/"
output_shapefile_path_tile_T32TLT = f"../raw_data/LANDKULT/data/tiles/T32TLT.shp"
output_shapefile_path_tile_T32TLS = f"../raw_data/LANDKULT/data/tiles/T32TLS.shp"
output_shapefile_path_tile_T32TMT = f"../raw_data/LANDKULT/data/tiles/T32TMT.shp"
output_shapefile_path_tile_T32TMS = f"../raw_data/LANDKULT/data/tiles/T32TMS.shp"
shapefile_path_landkult = '../raw_data/LANDKULT/data/LANDKULT_NUTZFL.shp'
output_shapefile_path_landkult = '../raw_data/LANDKULT/data/LANDKULT_NUTZFL_bern_bbox.shp'
output_shapefile_path_landkult_short = '../raw_data/LANDKULT/data/LANDKULT_NUTZFL_short_bern_bbox.shp'
output_shapefile_path_landkult_short_eodal = 'D:/Temp/AgroLuege/raw_data/LANDKULT/data/LANDKULT_NUTZFL_short_bern_bbox.shp'

In [ ]:
def convert_coordinates_to_utm(coordinates):
    utm_coordinates = []
    for lat, lon in coordinates:
        utm_coordinate = utm.from_latlon(lat, lon)
        utm_coordinates.append((utm_coordinate[0], utm_coordinate[1], utm_coordinate[2], utm_coordinate[3]))
    return utm_coordinates

def get_adjacent_points(utm_coordinates):
    adjacent_coordinates_beneath = []
    for easting, northing, zone_number, zone_letter in utm_coordinates:
        # Point beneath
        right_point = utm.to_latlon(easting - 100000, northing, zone_number, zone_letter)
        adjacent_coordinates_beneath.append(right_point)
    return adjacent_coordinates_beneath



In [ ]:
coordinates_T32TLT = [(6.3279, 46.8356), (7.8166, 46.8356), (7.8166, 47.8474), (6.3279, 47.8474)]
polygon_T32TLT = Polygon(coordinates_T32TLT)
gdf_T32TLT = gpd.GeoDataFrame(geometry=[polygon_T32TLT],crs="EPSG:4326").to_crs(crs="EPSG:32632")
gdf_T32TLT.to_file(output_shapefile_path_tile_T32TLT)

utm_coordinates_T32TLS = convert_coordinates_to_utm(coordinates_T32TLT)
coordinates_T32TLS = get_adjacent_points(utm_coordinates_T32TLS)
polygon_T32TLS = Polygon(coordinates_T32TLS)
gdf_T32TLS = gpd.GeoDataFrame(geometry=[polygon_T32TLS],crs="EPSG:4326").to_crs(crs="EPSG:32632")
gdf_T32TLS.to_file(output_shapefile_path_tile_T32TLS)

coordinates_T32TMT = [(7.6629, 46.8582), (9.1305, 46.8582), (9.1305, 47.8536), (7.6629, 47.8536)]
polygon_T32TMT = Polygon(coordinates_T32TMT)
gdf_T32TMT = gpd.GeoDataFrame(geometry=[polygon_T32TMT],crs="EPSG:4326").to_crs(crs="EPSG:32632")
gdf_T32TMT.to_file(output_shapefile_path_tile_T32TMT)

utm_coordinates_T32TMS= convert_coordinates_to_utm(coordinates_T32TMT)
coordinates_T32TMS = get_adjacent_points(utm_coordinates_T32TMS)
polygon_T32TMS = Polygon(coordinates_T32TMS)
gdf_T32TMS = gpd.GeoDataFrame(geometry=[polygon_T32TMS],crs="EPSG:4326").to_crs(crs="EPSG:32632")
gdf_T32TMS.to_file(output_shapefile_path_tile_T32TMS)

In [ ]:
# Define the coordinates
x1, y1, x2, y2 = 361630.,5140066., 416830.,  5238466.

# Create a GeoDataFrame with a single Point geometry
geometry = Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])
gdf_bern = gpd.GeoDataFrame(geometry=[geometry], crs="EPSG:32632")

# Save the GeoDataFrame to a shapefile
gdf_bern.to_file(output_shapefile_path_BERN)

In [ ]:
# # Read the shapefiles into GeoPandas DataFrames
# gdf1 = gpd.read_file(shapefile_path_landkult).to_crs('EPSG:32632')
# gdf2 = gpd.read_file(output_shapefile_path_BERN).to_crs('EPSG:32632')

# # Perform the intersection
# gdf1_only_beitrag = gdf1[gdf1['BEITRAG'] == 1]
# intersection_gdf_onlybeitrag = gpd.overlay(gdf1_only_beitrag, gdf2, how='intersection')

# intersection_gdf_onlybeitrag.to_file(output_shapefile_path_landkult, driver='ESRI Shapefile')
# intersection_gdf_onlybeitrag[0:100].to_file(output_shapefile_path_landkult_short, driver='ESRI Shapefile')

In [ ]:
# for i in [100,150,200,300,500,1000]:
#     intersection_gdf_onlybeitrag.sort_values('Shape_Area',ascending=False)[0:i].plot()
#     plt.show()
#     intersection_gdf_onlybeitrag.sort_values('Shape_Area',ascending=False)['Shape_Area'][0:i].hist()
#     plt.show()

# for i in [100,150,200,300,500,1000]:
#     # intersection_gdf.sort_values('Shape_Area',ascending=True)[0:i].plot()
#     # plt.show()
#     intersection_gdf_onlybeitrag.sort_values('Shape_Area',ascending=True)['Shape_Area'][0:i].hist()
#     plt.show()


# del intersection_gdf_onlybeitrag,gdf_bern,gdf1,gdf1_only_beitrag,gdf2

In [ ]:
def get_tile_folder_path(data_dir):
    paths = []
    def get_subdirectories(path, depth=0, max_depth=0):
        if depth > max_depth:
            return

        subdirectories = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
        for subdir in subdirectories:
            subdir_path = os.path.join(path, subdir)
            if subdir_path.endswith('.SAFE'):
                paths.append(subdir_path)
            get_subdirectories(subdir_path, depth + 1, max_depth)
    get_subdirectories(data_dir)
    # sort the paths by timestamp
    paths.sort()
    return paths


def make_polygon_within(polygon1, polygon2):
    return  polygon1.intersection(polygon2)


def read_tile_data_from_safe(tile_paths, coords, tile, band_selection = ['B02', 'B03', 'B04', 'B08'], polygon_size = 240*240):

    for path_SAFE in tile_paths[0:20]:
        for i,coordinates in coords.iterrows():
            x1, y1, x2, y2 = coordinates.iloc[:4].tolist()
            polygon = Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])
            polygon_gdf = gpd.GeoDataFrame(geometry=[polygon],crs='EPSG:32632')
            tile_shape_path = output_shapefile_path_tiles+f'{tile}.shp'
            tile_gdf = gpd.read_file(tile_shape_path)
            
            polygon_clipped_coords = make_polygon_within(tile_gdf.geometry[0],polygon_gdf.geometry[0])
            polygon_clipped_coords_df = gpd.GeoDataFrame(geometry=[polygon_clipped_coords],crs='EPSG:32632')
            intersection_area = polygon_clipped_coords_df.geometry.iloc[0].area

            if intersection_area == polygon_size:
                # print(f'started bounding_box: {coordinates} with index:{i} at timestamp {path_SAFE}')
                # print("Intersection Area:", polygon_clipped_coords_df.geometry.iloc[0].area)

                # read data from .SAFE dataset for the selected AOI and spectral bands
                handler = Sentinel2.from_safe(
                    in_dir=Path(path_SAFE),
                    vector_features=polygon_clipped_coords_df,
                    band_selection=band_selection,
                    apply_scaling=False
                )
                # ignore the value if its blackfilled
                if handler.is_blackfilled == True:
                    print(f"Skip is blackfilled: {path_SAFE}")
                    continue
                # first resample the spectral bands using bicubic interpolation
                handler.resample(
                    target_resolution=10,
                    interpolation_method=cv2.INTER_NEAREST_EXACT,
                    inplace=True
                )

                # create a numpy array and remove last band
                #TODO: check if this is okay, we select 24x24 but the outcome is 26x26
                # timestamp_tile_data = [handler.to_xarray().to_numpy()[0:4,0:24,0:24]]
                timestamp_tile_data = [handler.to_xarray().to_numpy()[0:4]]
                # save tile data
                save_tile_data(timestamp_tile_data, tile)
                save_ground_truth_data(i,tile)
            else:
                print("Intersection Area:", polygon_clipped_coords_df.geometry.iloc[0].area)
                print('edge case intersection not fully, is therefore ignored for this tile.')
                continue

def save_tile_data(temp_results_tensor, tile,dataset_data_name="data"):
    file_name_tile = f'../raw_data/BernCrop/tiles/{tile}.hdf5'
    tile= np.array(temp_results_tensor)
    data_shape = tile.shape
    print(data_shape)

    with h5py.File(file_name_tile, 'a') as hf:
        # Check if the dataset already exists
        if dataset_data_name in hf:
            dataset = hf[dataset_data_name]
        else:
            dtype = "float32"  # Use the appropriate data type for your data
            dataset = hf.create_dataset(dataset_data_name, shape=(0,) + data_shape[1:], dtype=dtype, maxshape=(None,) + data_shape[1:])
            
        current_size = dataset.shape[0]
        new_size = current_size + tile.shape[0]
        # Resize the dataset to accommodate the new batch
        dataset.resize(new_size, axis=0)
        # Append the new batch to the dataset
        dataset[current_size:new_size, :] = tile

def save_label_data(index_data,tile):
    # Specify the file path
    file_path = r'..\raw_data\BernCrop\tensor_label.pt'
    # Load the tensor from the file
    all_labels = torch.load(file_path)
    data_label = all_labels[index_data]

    save_tile_data(data_label,tile,'gt')

def save_field_label_data(index_data,tile):
    # Specify the file path
    file_path = r'..\raw_data\BernCrop\tensor_field.pt'

    # Load the tensor from the file
    all_labels = torch.load(file_path)
    data_label = all_labels[index_data]

    save_tile_data(data_label,tile,'gt_instance')

def save_ground_truth_data(index_data,tile):

    save_label_data(index_data,tile)
    save_field_label_data(index_data,tile)
    

def read_tile_data(tile,dataset_data_name="data"):
    filename_tile = f'../raw_data/BernCrop/tiles/{tile}.hdf5'
    # Open the HDF5 file in read mode
    with h5py.File(filename_tile, "r") as file:
        # Check if the "data" dataset exists in the file
        if dataset_data_name in file:
            # Access the dataset and read its contents into a NumPy array
            dataset = file[dataset_data_name][:]
        else:
            print(f"Dataset {dataset_data_name} not found in the HDF5 file.")
    return dataset

def get_coords():
    coords = pd.read_csv(r'..\raw_data\BernCrop\bboxes_sentinel_24x24_short.csv', index_col=None)
    coords= coords.iloc[:, 1:5]
    return coords


def stack_sample_tiles(tile_array,minimal_t=10,):
    print(minimal_t)
    if minimal_t != None:
        sampled_indices = np.random.choice(tile_array.shape[0], size=minimal_t, replace=False)
        sampled_elements_tile1 = tile_array[sampled_indices]
        print(sampled_elements_tile1.shape)
        tile_array = np.transpose(sampled_elements_tile1, (0, 2, 3, 1))
    tile_array_return = np.expand_dims(tile_array, axis=0)
    print(tile_array_return.shape)
    return tile_array_return


In [ ]:
# read tile data and save to dhf5 files
tiles = ['T32TLT','T32TLS','T32TMT','T32TMS']
coords = get_coords()

In [ ]:
for tile in tiles[1:2]:
# for tile in tiles:
    data_dir = Path('E:/S2_Data_CH22/' + tile)
    tile_paths = get_tile_folder_path(data_dir)
    read_tile_data_from_safe(tile_paths,coords,tile)

In [ ]:
tile1 = read_tile_data(tiles[1])
tile1_labels = read_tile_data(tiles[1],'gt')

In [ ]:
tile1.shape

In [ ]:
tile2 = tile1.copy()

In [ ]:
tile2= tile2[0:10]

In [ ]:
tile2.shape

In [ ]:
for i,v in enumerate(tiles):
    if i in [0,1]:
        tile = read_tile_data(tiles[i])
        tile_labels = read_tile_data(tiles[i],'gt')
        tile_field_labels = read_tile_data(tiles[i],'gt_instance')
        save_tile_data(stack_sample_tiles(tile),'BernCrop')
        save_tile_data(stack_sample_tiles(tile_labels,None),'BernCrop','gt')
        save_tile_data(stack_sample_tiles(tile_field_labels,None),'BernCrop','gt_instance')

In [ ]:
read_tile_data('BernCrop','gt').shape

In [ ]:
read_tile_data('BernCrop','gt_instance').shape